<a href="https://colab.research.google.com/github/anindyabecs/neural_network_diagrams/blob/main/Apollo_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import random
from st_aggrid import AgGrid, GridOptionsBuilder, GridUpdateMode, DataReturnMode

# 1. Page Configuration
st.set_page_config(
    page_title="Apollo Dashboard",
    page_icon="🚀",
    layout="wide"
)

# 2. Custom CSS for Styling
st.markdown("""
    <style>
    /* Center text in metrics */
    [data-testid="stMetric"] {
        width: fit-content;
        margin: auto;
    }
    [data-testid="stMetricValue"] {
        text-align: center;
        font-weight: bold;
    }
    [data-testid="stMetricLabel"] {
        text-align: center;
        font-weight: bold;
    }
    /* Custom styling for the Table */
    .stDataFrame {
        width: 100%;
    }
    </style>
""", unsafe_allow_html=True)


# 3. Mock Data Generator (Updated with new Columns)
@st.cache_data
def get_apollo_data():
    data = []
    statuses = ["Active", "Draft", "Deprecated", "Review"]
    sources = ["CRM DB", "Web Events", "IoT Gateway", "Legacy Mainframe", "Partner API"]
    # Types that map to the requested Transport Mediums
    types = ["JSON", "Avro", "Parquet", "XML", "CSV"]
    downstreams = ["Data Lake", "Fraud Engine", "Marketing Analytics", "Billing System"]
    processing_systems = ["Apollo Core", "Spark Cluster", "AWS Lambda", "Flink Stream"]
    transport_types_technical = ["SFTP", "HTTPS", "JMS", "S3 Bucket", "Kafka Topic"]
    delimiters = [",", "|", ";", "\\t", "Fixed Width"]

    # Database options
    db_types = ["Oracle 19c", "PostgreSQL 14", "MS SQL Server", "MongoDB", "Snowflake"]
    db_names = ["APOLLO_CORE", "CUST_DATA_LGR", "TRANS_HIST_V2", "REF_MASTER", "STAGING_DB"]

    # Generate 30 sample specs
    for i in range(1, 31):
        spec_name = f"Apollo_Spec_{chr(65 + (i%26))}_Ingest_{i}"
        source = random.choice(sources)
        dtype = random.choice(types)

        # Logic for file-specific fields
        is_file = dtype in ["CSV", "XML", "JSON", "Parquet"]

        # Logic for "Transport Medium" (XML, CSV, DB) request
        transport_medium = "N/A"
        current_db_type = "N/A"
        current_db_name = "N/A"

        if dtype == "XML":
            transport_medium = "XML"
        elif dtype == "CSV":
            transport_medium = "CSV"
        elif "DB" in source or "Mainframe" in source:
            transport_medium = "DB"
            current_db_type = random.choice(db_types)
            current_db_name = random.choice(db_names)
        else:
            # Fallback for others to match request roughly or assign random
            transport_medium = random.choice(["DB", "XML", "CSV"])
            if transport_medium == "DB":
                 current_db_type = random.choice(db_types)
                 current_db_name = random.choice(db_names)

        data.append({
            "Data Spec Name": spec_name,
            "Data Spec Description": f"Ingests {dtype} data from {source} for {random.choice(downstreams)}.",
            "Source": source,
            "Type": dtype,
            "Downstream": random.choice(downstreams),
            "Processing System": random.choice(processing_systems),
            "Service Endpoint": f"https://api.apollo.internal/v1/ingest/{1000 + i}",

            # Requested Details Pane Fields
            "Transport Medium": transport_medium,
            "Database Type": current_db_type,
            "Database Name": current_db_name,

            # Existing Columns
            "Transport Type": random.choice(transport_types_technical),
            "XML Path": f"/root/payload/data/v{i}" if transport_medium == "XML" else "N/A",
            "File Path": f"/mnt/apollo/landing/{spec_name.lower()}/" if is_file else "N/A",
            "File Name": f"batch_input_{1000+i}.{dtype.lower()}" if is_file else "N/A",
            "File Delimiter": random.choice(delimiters) if transport_medium == "CSV" else "N/A",

            # Keeping these for KPIs and Charts logic
            "Status": random.choice(statuses),
            "Rules Associated": random.randint(5, 65),
            "Spec ID": f"SPC-{1000 + i}"
        })
    return pd.DataFrame(data)


# Load data
df = get_apollo_data()

# --- HEADER SECTION ---
col_logo, col_title, col_btn = st.columns([1, 10, 2])

with col_logo:
    # Big Rocket Logo
    st.markdown("## 🚀")

with col_title:
    st.title("Apollo Application Monitor")
    st.write("Real-time oversight of Service Endpoints, Data Specs, and Rules.")

with col_btn:
    if st.button("Refresh Data", type="primary", icon=":material/refresh:"):
        st.rerun()

# --- KPI SECTION ---
with st.container():
    kpi1, kpi2, kpi3 = st.columns(3)

    # KPI 1: Service Endpoint (Count)
    with kpi1:
        with st.container(border=True):
            st.metric(
                label="Service Endpoints",
                value=len(df), # Dynamic count based on rows
                help="Total active service endpoints."
            )

    # KPI 2: Data Specs
    with kpi2:
        with st.container(border=True):
            active_specs = len(df[df['Status'] == 'Active'])
            st.metric(
                label="Data Specs",
                value=len(df),
                help="The data flow between systems."
            )

    # KPI 3: Total Rules
    with kpi3:
        with st.container(border=True):
            total_rules = df["Rules Associated"].sum()
            st.metric(
                label="Total Rules",
                value=total_rules,
                help="The total number of explicit rules set for the application."
            )

# --- FILTERS & DATA TABLE ---
st.subheader("📋 Data Specification Details")

# Filter Layout
f1, f2, f3 = st.columns([2, 2, 2])
with f1:
    search_query = st.text_input("Search Data Spec Name", placeholder="Type to search (e.g., Spec_C)...")
with f2:
    status_filter = st.multiselect("Filter by Status", df["Status"].unique(), default=["Active", "Draft"])

# Column Selection Logic (Visible in Grid)
all_columns_display = [
    "Data Spec Name", "Data Spec Description", "Source", "Type",
    "Downstream", "Processing System", "Service Endpoint", "Status",
    "Transport Medium", "Database Type"
]
default_columns = [
    "Data Spec Name", "Data Spec Description", "Source", "Type",
    "Downstream", "Processing System", "Service Endpoint", "Status"
]

with f3:
    selected_columns = st.multiselect(
        "Customize Table Columns",
        options=all_columns_display,
        default=default_columns,
        placeholder="Add/Remove columns..."
    )

# Apply Filters
filtered_df = df.copy()
if status_filter:
    filtered_df = filtered_df[filtered_df["Status"].isin(status_filter)]
if search_query:
    filtered_df = filtered_df[filtered_df["Data Spec Name"].str.contains(search_query, case=False)]

# --- MAIN LAYOUT (GRID) ---
with st.container(border=True):
    # Configure AgGrid
    gb = GridOptionsBuilder.from_dataframe(filtered_df)

    # Configure Selection to be Single Row (Clicking a row triggers selection)
    gb.configure_selection('single', use_checkbox=False)

    # Pagination Settings
    gb.configure_pagination(paginationAutoPageSize=False, paginationPageSize=10)

    # Hiding columns that aren't selected in the multiselect,
    # BUT keeping them in the dataframe so the Details Pane can access them.
    for col in filtered_df.columns:
        if col not in selected_columns:
            gb.configure_column(col, hide=True)
        else:
            gb.configure_column(col, hide=False)

    gb.configure_side_bar()
    gridOptions = gb.build()

    # Display Grid
    grid_response = AgGrid(
        filtered_df,
        gridOptions=gridOptions,
        enable_enterprise_modules=False,
        height=500,
        theme='streamlit',
        update_mode=GridUpdateMode.SELECTION_CHANGED,
        data_return_mode=DataReturnMode.FILTERED_AND_SORTED
    )

# --- POPUP DIALOG LOGIC ---

@st.dialog("🔍 Service Endpoint Details")
def show_endpoint_details(row):
    # Use native Streamlit components instead of HTML to prevent rendering issues
    st.caption("Service Endpoint Name")
    st.code(row.get('Service Endpoint', 'N/A'), language=None)

    st.divider()

    st.markdown("##### 📦 Transport Configuration")
    t_col1, t_col2 = st.columns(2)
    with t_col1:
        st.markdown("**Medium**")
        st.info(row.get('Transport Medium', 'N/A'))
        st.markdown("**File Type**")
        st.write(row.get('Type', 'N/A'))
    with t_col2:
        st.markdown("**File Name**")
        st.write(row.get('File Name', 'N/A'))
        st.markdown("**Delimiter**")
        st.code(row.get('File Delimiter', 'N/A'))

    st.divider()

    st.markdown("##### 🗄️ Database Info")
    d_col1, d_col2 = st.columns(2)
    with d_col1:
        st.markdown("**DB Type**")
        st.write(row.get('Database Type', 'N/A'))
    with d_col2:
        st.markdown("**DB Name**")
        st.write(row.get('Database Name', 'N/A'))

    st.divider

ModuleNotFoundError: No module named 'streamlit'